# Imports

In [1]:
import pandas as pd
import polars
from nba_api.stats import endpoints as nba_endpoints
from nba_api.live.nba import endpoints as nba_live_endpoints
import plotly.graph_objects as go
from math import floor, ceil
from PIL import Image

# Get Data

In [2]:
teamlogs = nba_endpoints.teamgamelogs
season = "2023-24"

In [3]:
all_games = teamlogs.TeamGameLogs(season_nullable=season).get_data_frames()[0]

In [4]:
# Calculate possessions for each game
all_games["POSS"]=.5 * (all_games["FGA"] + (.475*all_games["FTA"] - all_games["OREB"] + all_games["TOV"]))
# Identify home team
all_games["home_team"] = np.abs(all_games["MATCHUP"].str.contains("@").astype(int)-1)

In [5]:
# Separate home and away team rows
home_teams = all_games.loc[all_games["home_team"]==1]
away_teams = all_games.loc[all_games["home_team"]==0]
home_teams.columns = [str(col)+"_HOME" for col in home_teams.columns]
away_teams.columns = [str(col)+"_AWAY" for col in away_teams.columns]

In [6]:
# Merge opponents to home team rows
home_games = home_teams.merge(
    away_teams.assign()[["GAME_ID_AWAY", "POSS_AWAY", "PTS_AWAY", "TEAM_ABBREVIATION_AWAY"]],
    left_on="GAME_ID_HOME",
    right_on="GAME_ID_AWAY",
    how="inner"
)

In [7]:
# Calculate offensive/defensive ratings
home_games["OFF_RATING"] = 100/(home_games["POSS_HOME"]+home_games["POSS_AWAY"]) * home_games["PTS_HOME"]
home_games["DEF_RATING"] = 100/(home_games["POSS_HOME"]+home_games["POSS_AWAY"]) * home_games["PTS_AWAY"]

In [8]:
# Merge opponents to away team rows
away_games = away_teams.merge(
    home_teams.assign()[["GAME_ID_HOME", "POSS_HOME", "PTS_HOME", "TEAM_ABBREVIATION_HOME"]],
    left_on="GAME_ID_AWAY",
    right_on="GAME_ID_HOME",
    how="inner"
)

In [9]:
# Calculate offensive/defensive ratings
away_games["OFF_RATING"] = 100/(away_games["POSS_HOME"]+away_games["POSS_AWAY"]) * away_games["PTS_AWAY"]
away_games["DEF_RATING"] = 100/(away_games["POSS_HOME"]+away_games["POSS_AWAY"]) * away_games["PTS_HOME"]

In [10]:
sel_cols = [
    "GAME_ID_HOME", "GAME_DATE_HOME", "MATCHUP_HOME", "TEAM_ABBREVIATION_HOME", "TEAM_ABBREVIATION_AWAY",
    "OFF_RATING", "DEF_RATING", "POSS_HOME", "POSS_AWAY", "PTS_HOME", "PTS_AWAY"
]

upd_cols = [
    "GAME_ID", "GAME_DATE", "MATCHUP", "TEAM", "OPP", "OFF_RATING", "DEF_RATING", "POSS", "OPP_POSS", "PTS", "OPP_PTS"
]

In [11]:
# Select necessary columns
home_games_upd = home_games[
    [
        "GAME_ID_HOME", "GAME_DATE_HOME","MATCHUP_HOME", "TEAM_ABBREVIATION_HOME", "TEAM_ABBREVIATION_AWAY", 
        "OFF_RATING", "DEF_RATING", "POSS_HOME", "POSS_AWAY", "PTS_HOME", "PTS_AWAY"
    ]
]

home_games_upd.columns = upd_cols

In [12]:
# Select necessary columns
away_games_upd = away_games[
    [
        "GAME_ID_AWAY", "GAME_DATE_AWAY", "MATCHUP_AWAY", "TEAM_ABBREVIATION_AWAY", "TEAM_ABBREVIATION_HOME", 
        "OFF_RATING", "DEF_RATING","POSS_AWAY", "POSS_HOME", "PTS_AWAY", "PTS_HOME"
    ]
]

away_games_upd.columns = upd_cols

In [13]:
# Stack dataframes
games = pd.concat([
    home_games_upd, away_games_upd
], axis=0)

In [14]:
# Reset index
games = games.sort_values(["TEAM", "GAME_ID"]).reset_index().drop("index", axis=1)

In [15]:
# Set game number value to use for animation frames
games["GAME_NUM"] = games.groupby("TEAM").cumcount()+1

In [16]:
# Calculate cumulative stats
games["CUME_POSS"]=games.groupby("TEAM")["POSS"].cumsum()
games["CUME_OPP_POSS"]=games.groupby("TEAM")["OPP_POSS"].cumsum()
games["CUME_PTS"]=games.groupby("TEAM")["PTS"].cumsum()
games["CUME_OPP_PTS"]=games.groupby("TEAM")["OPP_PTS"].cumsum()

games["CUME_OFF_RATING"] = 100/(games["CUME_POSS"]+games["CUME_OPP_POSS"]) * games["CUME_PTS"]
games["CUME_DEF_RATING"] = 100/(games["CUME_POSS"]+games["CUME_OPP_POSS"]) * games["CUME_OPP_PTS"]
games["CUME_NET_RATING"] = games["CUME_OFF_RATING"] - games["CUME_DEF_RATING"]

In [17]:
games.head().transpose()

,0,1,2,3,4
GAME_ID,0022300018,0022300029,0022300039,0022300055,0022300063
GAME_DATE,2023-11-14T00:00:00,2023-11-17T00:00:00,2023-11-21T00:00:00,2023-11-28T00:00:00,2023-10-25T00:00:00
MATCHUP,ATL @ DET,ATL vs. PHI,ATL vs. IND,ATL @ CLE,ATL @ CHA
TEAM,ATL,ATL,ATL,ATL,ATL
OPP,DET,PHI,IND,CLE,CHA
OFF_RATING,121.577614,111.780294,134.751773,97.459102,102.79173
DEF_RATING,115.788204,121.416526,139.184397,118.807286,108.398552
POSS,51.675,51.125,55.65,54.7,54.3375
OPP_POSS,51.9625,52.65,57.15,53.0375,52.675
PTS,126,116,152,105,110


In [18]:
# Primary team colors for bars
TEAM_COLORS = {
    "ATL": "#E03A3E",
    "BOS": "#007A33",
    "BKN": "#000000",
    "CHA": "#1D1160",
    "CHI": "#CE1141",
    "CLE": "#860038",
    "DAL": "#00538C",
    "DEN": "#0E2240",
    "DET": "#C8102E",
    "GSW": "#1D428A",
    "HOU": "#CE1141",
    "IND": "#002D62",
    "LAC": "#C8102E",
    "LAL": "#552583",
    "MEM": "#5D76A9",
    "MIA": "#98002E",
    "MIL": "#00471B",
    "MIN": "#0C2340",
    "NOP": "#0C2340",
    "NYK": "#006BB6",
    "OKC": "#007AC1",
    "ORL": "#0077C0",
    "PHI": "#006BB6",
    "PHX": "#1D1160",
    "POR": "#E03A3E",
    "SAC": "#5A2D81",
    "SAS": "#C4CED4",
    "TOR": "#CE1141",
    "UTA": "#002B5C",
    "WAS": "#002B5C"
}

In [21]:
# make figure dictionary
fig_dict = {
    "data": [],
    "layout": {},
    "frames": []
}

# fill in most of layout
fig_dict["layout"]["xaxis"] = {
    "title": "Net Rating",
    # Setting static range so animation doesn't look choppy
    "range": [-25, 25],
    "showline": True,
    "linewidth": 0.5,
    "gridcolor": "lightgray"
}
fig_dict["layout"]["yaxis"] = {
    "categoryorder": "total ascending",
    "showticklabels": False,

}
fig_dict["layout"]["hovermode"] = "closest"
# You don't have to set this but if it's too small then not every bar will have a label
fig_dict["layout"]["height"] = 800
fig_dict["layout"]["plot_bgcolor"] = "white"
fig_dict["layout"]["showlegend"] = False

# There are many easing functions, but after testing most of them, 
#   the best for this use case is 'linear'
easing = "linear"

# Having the frame duration be slightly larger than the transition 
#   duration seems to produce smooth animations
frame_duration = 500
transition_duration = 450

# This structure is directly from the docs
fig_dict["layout"]["updatemenus"] = [
    {
        "buttons": [
            {
                "args": [
                    None,
                    {
                        "frame": {"duration": frame_duration, "redraw": True},
                        "fromcurrent": True,
                        "transition": {"duration": transition_duration, "easing": easing}
                    }
                ],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [
                    [None],
                    {
                        "frame": {"duration": 0, "redraw": True},
                        "mode": "immediate",
                        "transition": {"duration": 0}
                    }
                ],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }
]

sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 16},
        "prefix": "<b>Game #: ",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": transition_duration, "easing": easing},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

# Create initial chart view (Game 1)
game_num = 1
teams = games["TEAM"].unique()
for team in teams:
    dataset_by_game_and_team = games[
        (games["TEAM"] == team) &
        (games["GAME_NUM"] == game_num)
    ]
    data_dict = {
        "y": list(dataset_by_game_and_team["TEAM"]),
        "x": list(dataset_by_game_and_team["CUME_NET_RATING"]),
        "customdata": np.stack(
            (dataset_by_game_and_team["CUME_OFF_RATING"],
             dataset_by_game_and_team["CUME_DEF_RATING"]), axis=-1
        ),
        "type": "bar",
        "text": team,
        "marker_color": TEAM_COLORS.get(team),
        "textposition": "outside",
        "texttemplate": "<b>%{text}</b>",
        "hovertemplate": "<b>%{y}</b><br>"
                         "Net Rating: %{x:.2f}<br>"
                         "Offensive Rating: %{customdata[0]:.2f}<br>"
                         "Defensive Rating: %{customdata[1]:.2f}<br>"
                         "<extra></extra>",
        "orientation": "h",
        "name": team
    }
    fig_dict["data"].append(data_dict)

# Make a new frame for each game
for game in range(1, 83):
    frame = {"data": [], "name": str(game)}
    for team in teams:
        dataset_by_game_and_team = games[
            (games["TEAM"] == team) &
            (games["GAME_NUM"] == game)
        ]
        data_dict = {
            "y": list(dataset_by_game_and_team["TEAM"]),
            "x": list(dataset_by_game_and_team["CUME_NET_RATING"]),
            "customdata": np.stack(
                (dataset_by_game_and_team["CUME_OFF_RATING"],
                 dataset_by_game_and_team["CUME_DEF_RATING"]), axis=-1
            ),
            "type": "bar",
            "text": team,
            "marker_color": TEAM_COLORS.get(team),
            "textposition": "outside",
            "texttemplate": "<b>%{text}</b>",
            "hovertemplate": "<b>%{y}</b><br>"
                             "Net Rating: %{x:.2f}<br>"
                             "Offensive Rating: %{customdata[0]:.2f}<br>"
                             "Defensive Rating: %{customdata[1]:.2f}<br>"
                             "<extra></extra>",
            "orientation": "h",
            "name": team
        }
        frame["data"].append(data_dict)

    fig_dict["frames"].append(frame)
    
    slider_step = {"args": [
        [game],
        {"frame": {"duration": frame_duration, "redraw": True},
         "mode": "immediate",
         "transition": {"duration": transition_duration, "easing": easing}}
    ],
        "label": game,
        "method": "animate"}
    sliders_dict["steps"].append(slider_step)


fig_dict["layout"]["sliders"] = [sliders_dict]

fig = go.Figure(fig_dict)

fig.show()

In [22]:
fig.write_html("Net Rating Racing Bar Chart.html")

In [ ]:
game_num = 82

min_range = floor(min(
    games.loc[games["GAME_NUM"] == game_num, "CUME_OFF_RATING"].agg({"min", "max"}).loc["min"],
    games.loc[games["GAME_NUM"] == game_num, "CUME_DEF_RATING"].agg({"min", "max"}).loc["min"]
))
max_range = ceil(max(
    games.loc[games["GAME_NUM"] == game_num, "CUME_OFF_RATING"].agg({"min", "max"}).loc["max"],
    games.loc[games["GAME_NUM"] == game_num, "CUME_DEF_RATING"].agg({"min", "max"}).loc["max"]
))

axes_range = [min_range, max_range]
# axes_range = [100, 120]
line_location = (axes_range[0]+axes_range[1])/2

# Scatter Trace
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=games.loc[games["GAME_NUM"] == game_num, "CUME_OFF_RATING"],
        y=games.loc[games["GAME_NUM"] == game_num, "CUME_DEF_RATING"],
        mode="text",
        customdata=np.stack(
            (games.loc[games["GAME_NUM"] == game_num, "CUME_NET_RATING"],
            games.loc[games["GAME_NUM"] == game_num, "TEAM"]), axis=-1
        ),
        marker_color=[TEAM_COLORS.get(
            t) for t in games.loc[games["GAME_NUM"] == game_num, "TEAM"]],
        hovertemplate="<b>%{customdata[1]}</b><br>"
        "Net Rating: %{customdata[0]:.2f}<br>"
        "Offensive Rating: %{x:.2f}<br>"
        "Defensive Rating: %{y:.2f}<br>"
        "<extra></extra>",

    )
)

# Base Layouts
fig.update_layout(
    plot_bgcolor="#F4F3EE",
    height=1000,
    width=1000
)
fig.update_xaxes(
    gridcolor="#F4F3EE",
    range=axes_range
)
fig.update_yaxes(
    gridcolor="#F4F3EE",
    autorange="reversed",
    range=axes_range,
)

# x-axis
fig.add_annotation(
    x=min_range,
    y=line_location,
    xref="x", yref="y",
    text="",
    showarrow=True,
    axref="x", ayref='y',
    ax=max_range,
    ay=line_location,
    arrowhead=3,
    arrowwidth=1,
    arrowcolor="#0A0403"
)
fig.add_annotation(
    x=max_range,
    y=line_location,
    xref="x", yref="y",
    text="",
    showarrow=True,
    axref="x", ayref='y',
    ax=min_range,
    ay=line_location,
    arrowhead=3,
    arrowwidth=1,
    arrowcolor="#0A0403"
)
fig.add_annotation(
    x=max_range-1.5,
    y=line_location-.15,
    align="center",
    text=f"Offensive Efficiency",
    font=dict(size=14),
    showarrow=False,
)

# y-axis
fig.add_annotation(
    x=line_location,
    y=min_range,
    xref="x", yref="y",
    text="",
    showarrow=True,
    axref="x", ayref='y',
    ax=line_location,
    ay=max_range,
    arrowhead=3,
    arrowwidth=1,
    arrowcolor="#0A0403"
)
fig.add_annotation(
    x=line_location,
    y=max_range,
    xref="x", yref="y",
    text="",
    showarrow=True,
    axref="x", ayref='y',
    ax=line_location,
    ay=min_range,
    arrowhead=3,
    arrowwidth=1,
    arrowcolor="#0A0403"
)
fig.add_annotation(
    x=line_location+.2,
    y=max_range-1.5,
    align="center",
    text=f"Defensive Efficiency",
    font=dict(size=14),
    showarrow=False,
    textangle=-90
)

# diagonal line
fig.add_shape(
    type="line",
    # starting coordinates
    x0=min_range+0.5, y0=min_range+0.5,
    # ending coordinates
    x1=max_range-0.5, y1=max_range-0.5,
    # Make sure the points are on top of the line
    layer="below",
    # Style it like the axis lines
    line=dict(dash="dash", color="#C0C0C0", width=1)
)

# diagonal annotation -> positive/negative teams
fig.add_annotation(
    x=min_range*1.012, y=min_range*1.01,
    align="left",
    text=f"Positive Teams",
    font=dict(size=11, color="gray"),
    showarrow=False,
    textangle=45,
)
fig.add_annotation(
    x=min_range*1.0105, y=min_range*1.0125,
    align="left",
    text=f"Negative Teams",
    font=dict(size=11, color="gray"),
    showarrow=False,
    textangle=45
)

## quadrant annotations
lower_location = (line_location+min_range)/2
higher_location = (line_location+max_range)/2
#club thibs
fig.add_annotation(
    x=lower_location*.99, y=lower_location,
    align="center",
    text=f"Club<br>Thibs",
    font=dict(size=14, color="#82817D"),
    showarrow=False,
)
#quadrant of wow
fig.add_annotation(
    x=higher_location, y=lower_location,
    align="center",
    text=f"The Quadrant<br>of Wow",
    font=dict(size=14, color="#82817D"),
    showarrow=False,
)
#quadrant of woe
fig.add_annotation(
    x=lower_location*1.01, y=higher_location,
    align="center",
    text=f"The Quadrant<br>of Woe",
    font=dict(size=14, color="#82817D"),
    showarrow=False,
)
#club d'antoni
fig.add_annotation(
    x=higher_location*.99, y=higher_location,
    align="center",
    text=f"Club<br>D'Antoni",
    font=dict(size=14, color="#82817D"),
    showarrow=False,
)

# add team images
for t in games["TEAM"].unique():
    fig.add_layout_image(
        source=Image.open(f'/Users/jeremycolon/Documents/Personal/nba_logos/{t}.png'),
        sizex=1.5,
        sizey=1.5,
        name=t,
        xref="x",
        yref="y",
        x=games.loc[(games["GAME_NUM"] == game_num) & (games["TEAM"]==t), "CUME_OFF_RATING"].values[0],
        y=games.loc[(games["GAME_NUM"] == game_num) & (games["TEAM"]==t), "CUME_DEF_RATING"].values[0],
        layer="above",
        opacity=1, xanchor="center",yanchor="middle"
    )
    
fig.show()